A little notebook to help visualise the official numbers for personal use. Absolutely no guarantees are made.

**This is not a replacement for expert advice. Please listen to your local health authorities.**

The data is dynamically loaded from: https://github.com/CSSEGISandData/COVID-19 

Note: Aggregation levels in the US seems to have changed on 2020-03-10

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cartopy.crs as ccrs # needs Proj, which is not pip-installable
import matplotlib.animation as animation
# where is ffmpeg on your sytem (not pip-installable)?
plt.rcParams['animation.ffmpeg_path'] = '/opt/local/bin/ffmpeg'

In [ ]:
from jhu_helpers import *

In [ ]:
# get data
jhu, days = join_jhu_df(*get_jhu_data())

In [ ]:
# choose which metric to plot and how to scale it
plot_metric = 'new_in_21_days'
df          = jhu[plot_metric]
smax        = df[days].max().max() / 500

In [ ]:
# callback on each animation frame 
def update_map(day, df, ax, scatter, smax, plot_metric):
    ax.set_title(f'Confirmed {plot_metric.replace("_", " ").title()} on {day}')
    scatter.set_sizes(df[day].values / smax)
    return scatter,    

In [ ]:
plt.close(0)
fig = plt.figure(0, figsize=(9.6, 5.4))

# setup cartopy
ax = plt.axes(
    (0,0,1,.94),
    frameon=False,
    projection=ccrs.Mercator(
        central_longitude=0,  
        min_latitude=-60,
        max_latitude=75
    )
)
ax.stock_img()
ax.coastlines(color=(0,0,.1,.8))

# plot first frame
scatter = ax.scatter(
    df.Long.values, 
    df.Lat.values, 
    s=df[days[0]].values / smax, 
    facecolors=(1,0,0,.5), 
    edgecolors=(.8,0,0,1), 
    transform=ccrs.PlateCarree()
)

## setup & start animation
scatter_ani = animation.FuncAnimation(
    fig, update_map, days, 
    repeat=True, interval=10, blit=False,
    fargs=(df, ax, scatter, smax, plot_metric),
)

In [ ]:
#scatter_ani.event_source.stop() # stop animation

In [ ]:
#update_map(days[-1], df, ax, scatter, smax, plot_metric) # manually set frame

In [ ]:
## save as video
#FFwriter = animation.FFMpegWriter(fps=15)
#scatter_ani.save('map_animation.mp4', writer=FFwriter)

In [ ]:
## show saved video
#Video("map_animation.mp4", width=1000)

In [ ]:
# save as gif
#IMwriter = animation.ImageMagickWriter(fps=15)
#scatter_ani.save('map_animation.gif', writer=IMwriter)